In [1]:
import pandas as pd
import os
import numpy as np
import csv

# "제거 + 하나로 합침

In [ ]:
data_list = []
hap = 0
for filename in os.listdir('E:/폴더이름'):
    if filename.endswith(".txt"):
        df = pd.read_csv(f'E:/폴더이름/{filename}',encoding='utf-8',sep='|')
        count = len(df)
        hap += count
        print(len(df))
        df1 = pd.DataFrame()
        for i in df.columns:
            j=i
            if i=='점 가맹점블록코드':
                j='점 가맹점 블록코드'
            df1[j] = df[i].str.replace("''",'')
        data_list_list.append(df1)
        concat_df = pd.concat(data_list, ignore_index=True)
concat_df.head()

# 업종코드 병합

In [ ]:
upjong = pd.read_csv('E/:TB_SHCARD~0',encoding='utf-8-sig',sep='|')
upjong.head()

In [ ]:
for i in upjong.columns:
    j=i
    if i=='점 가맹점블록코드':
                j='점 가맹점 블록코드'
            upjong[j] = upjong[i].str.replace("''",'')

In [ ]:
upjong['SF_UPJONG_CD'] = upjong['SF_UPJONG_CD'].str.upper()
upjong.head()

In [ ]:
merge_df = pd.merge(concat_df, upjong, how='inner', left_on='업종코드',right_on='SF_UPJON_CD')
merge_df.head()

In [ ]:
merge_df = merge_df.drop(columns=['SF_L_UPJONG_CD','SF_M_UPJONG_CD','SF_UPJONG_CD'])
merge_df.head()

# 블록코드 병합

In [ ]:
shp=pd.read_excel('E:/shp(v3)/TBGIS_BLCK_RELM_INFO.xlsx')
shp.head()

In [ ]:
a = pd.read_csv('E:/TBBLOCK_TYPE_V3.txt',encoding='utf-8-sig',sep='|')
a.head()

In [ ]:
a = a.drop(columns=['블록유형소분류','배후지유형소분류'])
shpa = pd.merge(shp,a,how='inner',left_on='BLCK_CD',right_on='블록코드')

In [ ]:
merge_df['가맹점블록코드'] = merge_df['가맹점블록코드'].astype('int64')
merge_shp_df = pd.merge(merge_df,shpa,how='inner',left_on='가맹점블록코드',right_on='BLCK_CD')
merge_shp_df.columns

In [ ]:
merge_shp_df.to_csv('E/preprocessing_data.csv')

In [ ]:
merge_shp_df['일별']=pd.to_datatime(merge_shp_df['일별'],format='%Y%m%d')
merge_shp_df['요일']=merge_shp_df['일별'].dt.day_name()
merge_shp_df.head()

In [ ]:
merge_shp_df = merge_shp_df.drop(columns=['가맹점블록코드','업종코드','BLCK_CD'])

# 구,동 변환

In [ ]:
daedosi = pd.read_excel('E:/행정동및대도시권_코드_2017.xlsx')
daedosi.head()

In [ ]:
daedosi = dasdosi.drop(0)
daedosi['행자부행정동코드'] = daedosi['행자부행정동코드'].unique().astype('int64')

In [ ]:
merge_shp_dong_df = pd.merge(merge_shp_df, daedosi, how='inner',left_on='ADSTRD_CD',right_on='행자부행정동코드')
merge_shp_dong_df = merge_shp_dong_df.drop(columns=['통계청행정동코드','행자부행정동코드','시도명','ADSTRD_CD','SIGNGU_CD'])
merge_shp_dong_df.columns

# 상권코드-행정동코드

In [51]:
df = pd.read_csv('서울시 상권분석서비스(상권영역).csv',encoding='cp949')
df = df[df['상권_구분_코드_명']=='골목상권']

In [52]:
len(df['상권_코드'].unique())

1090

In [53]:
len(df['행정동_코드'].unique())

378

In [55]:
dong = pd.read_csv('데청캠자료/데이터/신사동분리버전행정동코드.csv')
dong

,ADSTRD_CD_x,ADSTRD_NM,ADSTRD_CD_y
0,1101053,사직동,11110530
1,1101054,삼청동,11110540
2,1101055,부암동,11110550
3,1101056,평창동,11110560
4,1101057,무악동,11110570
...,...,...,...
416,1125070,둔촌1동,11740690
417,1125071,둔촌2동,11740700
418,1125072,암사1동,11740570
419,1125073,천호2동,11740610


In [56]:
a = set(df[df['상권_구분_코드_명']=='골목상권']['행정동_코드'].unique())
b = set(dong['ADSTRD_CD_y'].unique())

only_df_dongs = list(b-a)
only_df_df = list(a-b)

# 주어진 행정동명을 포함하지 않은 행 추출
dong = dong[~dong["ADSTRD_CD_y"].isin(only_df_dongs)]
df = df[~df['행정동_코드'].isin(only_df_df)]

In [57]:
len(dong['ADSTRD_CD_y'].unique())

376

In [61]:
df['상권_코드']=df['상권_코드']-1000000

In [63]:
df

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,형태정보
0,202212,A,골목상권,2110008,배화여자대학교(박노수미술관),197093,453418,11110,11110515,NaN
1,202212,A,골목상권,2110009,자하문터널,196991,455057,11110,11110550,NaN
2,202212,A,골목상권,2110010,평창동서측,197064,456643,11110,11110560,NaN
3,202212,A,골목상권,2110017,정독도서관,198581,453781,11110,11110600,NaN
4,202212,A,골목상권,2110018,중앙고등학교,198883,453690,11110,11110600,NaN
...,...,...,...,...,...,...,...,...,...,...
1085,202212,A,골목상권,2110892,관악구 중앙길,195677,442761,11620,11620615,NaN
1086,202212,A,골목상권,2110893,관악소방서(문영여고),195802,441665,11620,11620595,NaN
1087,202212,A,골목상권,2110894,관악동부센트레빌아파트(은천로35라길),195881,443185,11620,11620565,NaN
1088,202212,A,골목상권,2110966,논현2동주민센터,203386,446575,11680,11680531,NaN


In [64]:
df.to_csv('상권코드_행정동코드.csv',index=False)

In [48]:
dong.to_csv('7,8자리행정동코드.csv',index=False)